# Notation and description

Our homology inference pipeline for a manifold X using adaptive samples and persistence has the following parameters, which are either "fixed" or "free"

## Fixed: 
    - R, which is less than the reach of the space X 
    - delta, how close sample points are to X
    - E_lfs, the maximum error between estimated local feature sizes and actual local feature sizes    
    
## Free: 
    - lambda, where lambda * R is the initial density of the unsparsified sample
    - mu, which is the proportion of the local feature size to use while adaptively subsampling
    - a, which is a free variable
    
For a fixed set of values for the fixed parameters, valid combinations of the free parameters are those which satisfy two inequalities. One of these inequalities is linear in a. The other is a rational polynomial function in a, lambda, and mu. In fact, the second inequality is more easily computed by making a substitution lplusmu = lambda + (1+delta/R) * mu.

We fix computed values for the reach and reasonable error values for the fixed error parameters. Then we naively search for valid combinations (a,lplusmu) by constructing a grid of possible parameter values in R^2 and evaluating the inequalities.

In [1]:
# Fixed error parameters
E_lfs = 1e-10
base_delta = 1e-10


# Base functions
function beta_prime(lplusmu,delta,R)
    delta_R = delta/R
    return lplusmu+3*delta_R
end

function alpha_prime(lplusmu,delta,R)
    delta_R = delta/R
    return delta_R/beta_prime(lplusmu,delta,R)
end

function M_k(delta,R,E=E_lfs)
    delta_R = delta/R
    return E/R + delta_R + 1
end

function M_k_hat(delta,R,E=E_lfs)
    delta_R = delta/R
    return E/(R*(1-delta_R)^2) + 1/(1-delta_R)
end

function a_prime(a,alpha,beta)
    return (a-alpha*beta)*(1-beta) - beta
end

function b_prime(b,alpha,beta)
    return (b+alpha*beta)/(1 - 2*(b+alpha*beta))
end

# Functions for bounds
function lower_bound(alpha,beta,a,b)
    ap = a_prime(a,alpha,beta)
    bp = b_prime(b,alpha,beta)
    return (1-ap)^2 + ( (bp-ap) + ( (b*(1+2*bp-ap))/(1-(b+alpha*beta)) ))^2
end

function upper_bound(alpha,beta,a,b)
    ap = a_prime(a,alpha,beta)
    bp = b_prime(b,alpha,beta)
    return (1 - ( (alpha*beta*(1+2*bp-ap))/(1 - (b+beta)) ) )^2
end


upper_bound (generic function with 1 method)

In [2]:
function construct_grid_points(initial,final,number)
    step_size = (final-initial)/(number-1)
    return [initial + i*step_size for i in 0:(number-1)]
end

construct_grid_points (generic function with 1 method)

In [3]:
number_of_points = 500
# These bounds cover the range of possible valid values
a_points = construct_grid_points(1e-4/4,0.22/4,number_of_points)
lambda_plus_mu_points = construct_grid_points(1e-4,5e-2,number_of_points)

function assertion_checker(a,lplusmu,delta,R)
    MK = M_k(delta,R)
    MKH = M_k_hat(delta,R)
    b = 4*(MK*MKH)^2*a
    if (1/3) - delta/R <= b
        return false
    end
    alphprime = alpha_prime(lplusmu,delta,R)
    betaprime = beta_prime(lplusmu,delta,R)
    return lower_bound(alphprime,betaprime,a,b) < upper_bound(alphprime,betaprime,a,b)
end

the_grid = Base.product(a_points,lambda_plus_mu_points)


butterfly_R = 0.103
print(M_k(base_delta,butterfly_R)," ",M_k_hat(base_delta,butterfly_R))
function curried_checker(a,lplusmu) 
    return assertion_checker(a,lplusmu,base_delta,butterfly_R)
end


1.0000000019417477 1.0000000019417477

curried_checker (generic function with 1 method)

In [4]:
valid_choices = [combination for combination in the_grid if curried_checker(combination[1],combination[2])]



9121-element Array{Tuple{Float64,Float64},1}:
 (0.00013517034068136273, 0.0001)
 (0.00024534068136272544, 0.0001)
 (0.0003555110220440882, 0.0001)
 (0.00046568136272545093, 0.0001)
 (0.0005758517034068136, 0.0001)
 (0.0006860220440881764, 0.0001)
 (0.0007961923847695391, 0.0001)
 (0.0009063627254509018, 0.0001)
 (0.0010165330661322646, 0.0001)
 (0.0011267034068136274, 0.0001)
 (0.00123687374749499, 0.0001)
 (0.0013470440881763529, 0.0001)
 (0.0014572144288577155, 0.0001)
 ⋮
 (0.012364078156312627, 0.006500000000000001)
 (0.01247424849699399, 0.006500000000000001)
 (0.012584418837675353, 0.006500000000000001)
 (0.012694589178356714, 0.006500000000000001)
 (0.012804759519038077, 0.006500000000000001)
 (0.01291492985971944, 0.006500000000000001)
 (0.013025100200400803, 0.006500000000000001)
 (0.013135270541082166, 0.006500000000000001)
 (0.013245440881763527, 0.006500000000000001)
 (0.01335561122244489, 0.006500000000000001)
 (0.013465781563126254, 0.006500000000000001)
 (0.01357595190380

In [5]:
a_coordinates = [coordinate[1] for coordinate in valid_choices]
lambda_plus_mu_coordinates = [coordinate[2] for coordinate in valid_choices]
using Plots
#display(plot(a_coordinates, lambda_plus_mu_coordinates, seriestype = :scatter))

In [6]:
# Larger lplusmu values are easier for us to work with
valid_choices[findmax(lambda_plus_mu_coordinates)[2]]

(0.011152204408817637, 0.006500000000000001)

In [8]:
# We now have a line of solutions lambda + (1+delta/R) * mu = 0.0065
# The sample for which we computed lfs approximations has lambda = 0.0019
mu = (0.0065 - 0.0019)/(1+(base_delta/butterfly_R))

0.00459999999553398